In [3]:
import time
load("kraus_tables.sage")
DB = CremonaDatabase(); DB
p = 3
kraus = kraus_table_Q3

In [4]:
objfile = os.path.join('./', '3-ktype_mtype_data.sobj')
# save(label_type_tuples, objfile)
label_type_tuples = load(objfile)  # format: (cremona label (string), Kodaira type (Sage custom type), integer)

In [6]:
# scratch
label_type_tuples[0:10]

[('36a1', III, 4),
 ('36a2', III, 4),
 ('36a3', III*, 4),
 ('36a4', III*, 4),
 ('54a1', IV*, 12),
 ('54a2', II*, 12),
 ('54a3', II, 12),
 ('54b1', II, 12),
 ('54b2', IV*, 12),
 ('54b3', IV, 12)]

In [7]:
objfile = os.path.join('./', '3-ktype_mtype_first_examples.sobj')
# save(first_examples, objfile)
first_examples = load(objfile)

In [8]:
ktype_given_mtype = dict.fromkeys([2, 3, 4, 6, 8, 12, 24], [])
for (_, ktype, mtype) in label_type_tuples:
    if ktype not in ktype_given_mtype[mtype]:
        ktype_given_mtype[mtype] = ktype_given_mtype[mtype] + [ktype]
ktype_given_mtype

{2: [I0*],
 3: [II, II*],
 4: [III, III*],
 6: [IV, IV*],
 8: [],
 12: [IV*, II*, II, IV],
 24: []}

In [9]:
first_examples

[('36a1', III, 4),
 ('36a3', III*, 4),
 ('54a1', IV*, 12),
 ('54a2', II*, 12),
 ('54a3', II, 12),
 ('54b3', IV, 12),
 ('126a1', I0*, 2),
 ('162a1', IV, 6),
 ('162a2', IV*, 6),
 ('162b1', II, 3),
 ('162b2', II*, 3)]

In [10]:
E = EllipticCurve('162b1'); E

Elliptic Curve defined by y^2 + x*y + y = x^3 - x^2 - 5*x + 5 over Rational Field

In [11]:
kraus(E)

3

In [ ]:
## attempt to compute monodromy order by another method leads to stack overflow after a few (couple?) hours
#t0 = time.time()
#E = EllipticCurve('162b1'); print(E)
#print(E.local_data(3))
#print(E.c4())
#F.<b> = E.division_field(5); print(F)
#pp = F.prime_above(3); print(pp.ramification_index())
#t1 = time.time()
#print("time: {}".format(t1-t0))

In [10]:
## PASTED FROM p=2 WORKSHEET

# remarks:
# McCallum proves exponent of inertial monodromy kills component group
# exponents: 2, 3, 4, 6, 12, |-> 2, 3, 4, 6, 4, 12
#
# for C_2, the possible component group are: 0, C_2, C_2 x C_2,
#        but we don't see C_2
# note: in this case, see case monodromy group smaller than component group
# note: I think Dokchitser paper shows this is the only situation where potentially ordinary
#       reduction is possible.
#
# for C_3, possibilities are 0 and C_3, 
#        but we don't see 0
#
# for C_4, possibilities are 0, C_2, C_4, C_2 x C_2,
#        but we ONLY see one of these, C_2
#
# for C_6, possibilities are 0, C_2, C_3, C_2 x C_2,
#        but we don't see C_2 or C_3
# note: C_2 x C_2 is not a subquotient of C_6
# note: Dokchitsers show this is potentially supersingular
#
# for Q_8, the possiblities are 0, C_2, C_4, C_2 x C_2, and we see all of these
# 
# for SL_2(F_3), the possiblities are 0, C_2, C_4, C_2 x C_2, and we see all of these

In [69]:
##########
## computing size of automorphism groups of first examples
##########

In [16]:
# by Silverman Theorem 10.1: 
# Aut of generic fiber is always C_2
# Aut of the special fiber is C_2 iff 3 \nmid j, otherwise is order 12 (it is the dicyclic group)
# We see that, in our list of examples (must look at more, not just first occurances), 
helper = {0:12, 1:2, 2:2}
for (label, ktype, mtype) in first_examples:
    E = EllipticCurve(label)
    j = E.j_invariant()
    print label, "$", latex(ktype),"$", mtype, helper[mod(j,3)] , "\\\\"

 36a1 $ III $ 4 12 \\
36a3 $ III^{*} $ 4 12 \\
54a1 $ IV^{*} $ 12 12 \\
54a2 $ II^{*} $ 12 12 \\
54a3 $ II $ 12 12 \\
54b3 $ IV $ 12 12 \\
126a1 $ I_0^{*} $ 2 2 \\
162a1 $ IV $ 6 12 \\
162a2 $ IV^{*} $ 6 12 \\
162b1 $ II $ 3 12 \\
162b2 $ II^{*} $ 3 12 \\


In [29]:
def ktype_to_cgroup_order(ktype):
    s = str(ktype)
    if ktype in [KodairaSymbol("I0"), KodairaSymbol("I1"), KodairaSymbol("II"), KodairaSymbol("II*")]:
        return 1
    elif ktype in [KodairaSymbol("III"), KodairaSymbol("III*")]:
        return 2
    elif ktype in [KodairaSymbol("IV"), KodairaSymbol("IV*")]:
        return 3
    elif s[-1] == '*':   #I_n^* cases (anticipating group output in another function)
        n = int(s[1:-1])
        if n%2 == 0:
            return 4
        else:
            return 4
    else:                # we are in case I_n
        n = int(s[1:])
        return n

def ktype_to_cgroup(ktype):
    s = str(ktype)
    if ktype in [KodairaSymbol("I0"), KodairaSymbol("I1"), KodairaSymbol("II"), KodairaSymbol("II*")]:
        return CyclicPermutationGroup(1)
    elif ktype in [KodairaSymbol("III"), KodairaSymbol("III*")]:
        return CyclicPermutationGroup(2)
    elif ktype in [KodairaSymbol("IV"), KodairaSymbol("IV*")]:
        return CyclicPermutationGroup(3)
    elif s[-1] == '*':   #I_n^* cases (anticipating group output in another function)
        n = int(s[1:-1])
        if n%2 == 0:
            return KleinFourGroup()
        else:
            return CyclicPermutationGroup(4)
    else:                # we are in case I_n
        n = int(s[1:])
        return CyclicPermutationGroup(n)

def ktype_to_cgroup_gid(ktype):
    s = str(ktype)
    if ktype in [KodairaSymbol("I0"), KodairaSymbol("I1"), KodairaSymbol("II"), KodairaSymbol("II*")]:
        return [1,1]
    elif ktype in [KodairaSymbol("III"), KodairaSymbol("III*")]:
        return [2,1]
    elif ktype in [KodairaSymbol("IV"), KodairaSymbol("IV*")]:
        return [3,1]
    elif s[-1] == '*':   #I_n^* cases (anticipating group output in another function)
        n = int(s[1:-1])
        if n%2 == 0:
            return [4,2]
        else:
            return [4,1]
    else:                # we are in case I_n
        n = int(s[1:])
        return [n,1]
    
## to get useful name, use gap.StructureDescription 


def ktype_to_stab_index(ktype):
    s = str(ktype)
    if ktype in [KodairaSymbol("II"), KodairaSymbol("II*")]:
        return 6
    elif ktype in [KodairaSymbol("III"), KodairaSymbol("III*")]:
        return 4
    elif ktype in [KodairaSymbol("IV"), KodairaSymbol("IV*")]:
        return 3
    elif s[-1] == '*':   #I_n^* cases (anticipating group output in another function)
        return 2
    else:                # we are in case I_n
        raise ValueError("Kodaira type conversion to stabilization index not implemented.")

In [23]:
# which curves have CM?
filter(lambda t: EllipticCurve(t[0]).has_cm(), first_examples)

[('36a1', III, 4), ('36a3', III*, 4)]

In [24]:
filter(lambda t: t[2]==4, first_examples)

[('36a1', III, 4), ('36a3', III*, 4)]

In [26]:
# is there a potentially ordinary curve?
E = EllipticCurve('126a1')
K1.<b> = NumberField(x^2-3)
EK1 = E.base_extend(K1)
pp1 = K1.prime_above(p)
print(EK1.local_data(pp1))
Ekappa1 = EK1.reduction(pp1); 
"Base change has ordinary reduction: {}".format(Ekappa1.is_ordinary())

Local data at Fractional ideal (b):
Reduction type: good
Local minimal model: Elliptic Curve defined by y^2 + b*x*y + b*y = x^3 + (-1)*x^2 + (-2)*x + (-1) over Number Field in b with defining polynomial x^2 - 3
Minimal discriminant valuation: 0
Conductor exponent: 0
Kodaira Symbol: I0
Tamagawa Number: 1


'Base change has ordinary reduction: True'

In [27]:
# do the Dicy12 curves become type II after tame base change (the only C3 curves I see are II/II*)
# answer: No.
L = ['54a1','54a2', '54a3', '54b3']
for label in L:
    E = EllipticCurve(label)
    K1.<b> = NumberField(x^4-p)
    K1.<b> = K1.galois_closure()
    EK1 = E.base_extend(K1)
    pp1 = K1.prime_above(p)
    print(EK1.local_data(pp1))
    print('')

Local data at Fractional ideal (3, 1/360*b^5 - 41/120*b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + (-1/900*b^6+1/144*b^4+11/300*b^2+7/48)*x*y + (1/72*b^4+7/24)*y = x^3 + (-1/1200*b^6-39/400*b^2-1/2)*x^2 + (1/1200*b^6+39/400*b^2-1/2)*x + 1 over Number Field in b with defining polynomial x^8 + 42*x^4 + 5625
Minimal discriminant valuation: 12
Conductor exponent: 6
Kodaira Symbol: IV*
Tamagawa Number: 3

Local data at Fractional ideal (3, 1/360*b^5 - 41/120*b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + (1/360*b^5-41/120*b)*x*y + (1/180*b^5+19/60*b)*y = x^3 + (1/1200*b^6+39/400*b^2-9/2)*x + (1/150*b^6-11/50*b^2) over Number Field in b with defining polynomial x^8 + 42*x^4 + 5625
Minimal discriminant valuation: 8
Conductor exponent: 6
Kodaira Symbol: IV
Tamagawa Number: 3

Local data at Fractional ideal (3, 1/360*b^5 - 41/120*b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + x

In [26]:
# do the IV/IV* type-12 curves become type 3 after quadratic base change? Yes.
L = ['54a3','54a2']
for label in L:
    E = EllipticCurve(label)
    K1.<b> = QuadraticField(p)
    EK1 = E.base_extend(K1)
    pp1 = K1.prime_above(p)
    print(EK1.local_data(pp1))
    print()

Local data at Fractional ideal (b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + x*y = x^3 + (-1)*x^2 + (-3)*x + 3 over Number Field in b with defining polynomial x^2 - 3
Minimal discriminant valuation: 6
Conductor exponent: 4
Kodaira Symbol: IV
Tamagawa Number: 3
()
Local data at Fractional ideal (b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + b*x*y + b*y = x^3 + (-15)*x + (-30) over Number Field in b with defining polynomial x^2 - 3
Minimal discriminant valuation: 10
Conductor exponent: 4
Kodaira Symbol: IV*
Tamagawa Number: 1
()
Local data at Fractional ideal (b):
Reduction type: bad additive
Local minimal model: Elliptic Curve defined by y^2 + x*y = x^3 + (-1)*x^2 + (-3)*x + 3 over Number Field in b with defining polynomial x^2 - 3
Minimal discriminant valuation: 6
Conductor exponent: 4
Kodaira Symbol: IV
Tamagawa Number: 3
()
Local data at Fractional ideal (b):
Reduction type: bad additive
Local minimal mo

In [33]:
from tabulate import tabulate

aut_helper = {0:24, 1:2, 2:2}

rows = []
for (label, ktype, mtype) in first_examples:
    E = EllipticCurve(label)
    LD = E.local_data(p)
    stab_index = ktype_to_stab_index(LD.kodaira_symbol())  
    j = E.j_invariant()
    if ktype_to_cgroup_order(ktype) > 0:
        ktype_str_desc = gap.StructureDescription(ktype_to_cgroup(ktype)) #to get group name
        rows += [[label, LD.conductor_valuation(), mtype, LD.kodaira_symbol(), stab_index, ktype_str_desc,
                  E.tamagawa_number(p), aut_helper[mod(j,p)], E.has_cm()]]
print(tabulate(rows, headers=['label','f','M','Kod','stab','Phi','Tam','Aut','CM?']))
print(latex(tabulate(rows, headers=['label','f','M','Kod','stab','Phi','Tam','Aut','CM?'])))

label      f    M  Kod      stab  Phi        Tam    Aut  CM?
-------  ---  ---  -----  ------  -------  -----  -----  -----
36a1       2    4  III         4  C2           2     24  True
36a3       2    4  III*        4  C2           2     24  True
54a1       3   12  IV*         3  C3           3     24  False
54a2       3   12  II*         6  1            1     24  False
54a3       3   12  II          6  1            1     24  False
54b3       3   12  IV          3  C3           3     24  False
126a1      2    2  I0*         2  C2 x C2      2      2  False
162a1      4    6  IV          3  C3           3     24  False
162a2      4    6  IV*         3  C3           3     24  False
162b1      4    3  II          6  1            1     24  False
162b2      4    3  II*         6  1            1     24  False
\begin{array}{l}
\text{\texttt{label{ }{ }{ }{ }{ }{ }f{ }{ }{ }{ }M{ }{ }Kod{ }{ }{ }{ }{ }{ }stab{ }{ }Phi{ }{ }{ }{ }{ }{ }{ }{ }Tam{ }{ }{ }{ }Aut{ }{ }CM?}}\\
\text{\texttt{{-}{-}{